### Dependencias iniciales

In [14]:
!pip install requests
!pip install lxml

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   --------------- ------------------------ 1.6/4.0 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 11.4 MB/s  0:00:00


In [32]:
import requests
import json
from lxml import html
import time

### Configuración inicial para la solicitud

Test de scraping para **https://www.eluniversal.com.mx** 

In [4]:
URL_BASE = "https://api.queryly.com/json.aspx"

PARAMS =  {
    "queryly_key": "000ca78dce0f4c6b",
    "query": "Trump",
    "endindex": 0,
    "batchsize": 100,
    "callback": "searchPage.resultcallback",
    "showfaceted": "true",
    "extendeddatafields": "creator,imageresizer,promo_image,subheadline",
    "timezoneoffset": 360,
    "facetedkey": "pubDate|",
    "facetedvalue": "8760|"
}

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/141.0.0.0 Safari/537.36 OPR/125.0.0.0",
    "Accept": "*/*",
    "Referer": "https://www.eluniversal.com.mx/",
    "Accept-Language": "en-US,en;q=0.9,es;q=0.8"
}

### Generación de la solicitud inicial

In [5]:
response = requests.get(
    URL_BASE,
    params=PARAMS,
    headers=HEADERS,
    timeout=10
)

print(response.status_code)

200


Se recibe un json con padding en la respuesta, hay que eliminar dicho espaciado para usar el json original

In [6]:
text = response.text

json_str = text[text.find("(") + 1 : text.rfind(");")]

data = json.loads(json_str)

data.keys()

dict_keys(['metadata', 'faceted', 'related', 'items', 'ads'])

Dentro de items se encuentran los metadatos de las noticias a extraer

In [7]:
noticias = []

for item in data["items"]: 
    
    noticia = {
        "titulo": item.get("title"),
        "url": "https://www.eluniversal.com.mx" + item.get("link"),
        "author": item.get("creator"),
        "pubdate": item.get("pubdate"),
        "descripcion": item.get("description")
    }

    noticias.append(noticia)

In [8]:
test_url = noticias[0]["url"]

test_url

'https://www.eluniversal.com.mx/mundo/trump-afirma-que-china-devorara-a-canada-rechaza-votacion-a-favor-de-negocios-entre-ambos-paises/'

In [23]:
articulo = requests.get(
    test_url,
    headers=HEADERS,
    timeout=10
)

articulo.encoding = "utf-8"
articulo.status_code

200

In [ ]:
tree = html.fromstring(articulo.text)

articulo_subtitle = tree.xpath('//div[contains(@class,"sc__header")]//h2[contains(@class,"subTitle")]/text()')
articulo_content = tree.xpath('//div[contains(@class, "sc") and contains(@class, "pl-3")]')

section = articulo_content[0].xpath('.//section')[0]
parrafos = section.xpath('.//p[@itemprop="description" and contains(@class,"sc__font-paragraph")]')

# Para quitar el último parrafo de firma editorial (Creo que eso es)
parrafos = parrafos[:-1] 

contenido = [p.text_content().strip() for p in parrafos if p.text_content().strip() 
             and not p.text_content().strip().lower().startswith("lee también")
             and not p.text_content().strip().lower().startswith("únete a nuestro canal")]

articulo_texto = " ".join(contenido)

articulo_texto

'El presidente estadounidense Donald Trump afirmó que China devorará a Canadá. En un mensaje en Truth Social, el mandatario posteó que "Canadá se opone a la construcción de la cúpula Dorada sobre Groenlandia, a pesar de que protegería a Canadá". Mencionó que Canadá "votó a favor de negocios con China, ¡que los devorará en el primer año". Trump retiró este jueves la invitación que había hecho al primer ministro de Canadá, Mark Carney, para que fuera parte de la Junta de la Paz para Gaza que su gobierno impulsa y que otros 35 jefes de Estado han aceptado formar parte. "Estimado Primer Ministro Carney: Por la presente, le comunico que la Junta de la Paz retira la invitación para que Canadá se una a la que será la Junta de Líderes más prestigiosa jamás reunida", escribió Trump en su cuenta oficial de Truth Social. También el jueves, Carney respondió a Trump que Canadá "no existe gracias a Estados Unidos". "Canadá y Estados Unidos han forjado una asociación extraordinaria", afirmó Carney en

In [33]:
noticias = []

for item in data["items"]:
    
    url = "https://www.eluniversal.com.mx" + item.get("link")

    try:
        articulo = requests.get(
            url,
            headers=HEADERS,
            timeout=10
        )

        articulo.encoding = "utf-8"
            
        tree = html.fromstring(articulo.text)

        articulo_subtitle = tree.xpath('//div[contains(@class,"sc__header")]//h2[contains(@class,"subTitle")]/text()')
        articulo_content = tree.xpath('//div[contains(@class, "sc") and contains(@class, "pl-3")]')

        section = articulo_content[0].xpath('.//section')[0]
        parrafos = section.xpath('.//p[@itemprop="description" and contains(@class,"sc__font-paragraph")]')
        parrafos = parrafos[:-1]

        contenido = [p.text_content().strip() for p in parrafos if p.text_content().strip() 
             and not p.text_content().strip().lower().startswith("lee también")
             and not p.text_content().strip().lower().startswith("únete a nuestro canal")]
        
        articulo_texto = " ".join(contenido)

        noticia = {
            "titulo": item.get("title"),
            "subtitulo": articulo_subtitle,
            "url": url,
            "author": item.get("creator"),
            "pubdate": item.get("pubdate"),
            "contenido": articulo_texto
        }

        noticias.append(noticia)

        # Delay para no saturar
        time.sleep(1.5)

    except Exception as e:
        print(f"Error en {url}: {e}")
        continue

In [34]:
with open("noticias.json", "w", encoding="utf-8") as f:
    json.dump(
        noticias,
        f,
        ensure_ascii=False,  
        indent=2             
    )